In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications import preprocess_input
import cv2


data_path = "./Database"
image_path = os.path.join(data_path, "Image")
label_file = os.path.join(data_path, "Label.txt")


def load_labels(label_path):
    labels = pd.read_csv(label_path, sep="\t", header=None, names=["Image", "Pitch", "Roll", "Depth"])
    return labels


def load_images(labels_df, image_folder):
    images = []
    labels = []
    
    for _, row in labels_df.iterrows():
        img_file = os.path.join(image_folder, row["Image"])
        if os.path.exists(img_file):
            img = cv2.imread(img_file)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (224, 224))  
            images.append(img)
            labels.append(row[1:].values)  
        
    return np.array(images), np.array(labels)


labels_df = load_labels(label_file)
images, labels = load_images(labels_df, image_path)


images = images / 255.0


X = images
y_pose = labels[:, :2]  
y_depth = labels[:, 2]  


y_pose_class = to_categorical(y_pose)


X_train, X_temp, y_train_pose, y_temp_pose, y_train_depth, y_temp_depth = train_test_split(
    X, y_pose_class, y_depth, test_size=0.3, random_state=42)

X_val, X_test, y_val_pose, y_test_pose, y_val_depth, y_test_depth = train_test_split(
    X_temp, y_temp_pose, y_temp_depth, test_size=0.5, random_state=42)


ImportError: cannot import name 'preprocess_input' from 'tensorflow.keras.applications' (c:\Users\islas\OneDrive\Documents\GitHub\deep_learning_proj\.conda\Lib\site-packages\keras\_tf_keras\keras\applications\__init__.py)

In [3]:
def create_cnn(input_shape, num_classes=None, task_type='classification'):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
    ])
    
    if task_type == 'classification':
        model.add(Dense(num_classes, activation='softmax'))
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    else:
        model.add(Dense(1, activation='linear'))
        model.compile(optimizer='adam', loss='mse', metrics=['mse'])
    
    return model



In [4]:
def create_lenet(input_shape, num_classes=None, task_type='classification'):
    model = Sequential([
        Conv2D(6, (5, 5), activation='tanh', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Conv2D(16, (5, 5), activation='tanh'),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(120, activation='tanh'),
        Dense(84, activation='tanh'),
    ])
    
    if task_type == 'classification':
        model.add(Dense(num_classes, activation='softmax'))
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    else:
        model.add(Dense(1, activation='linear'))
        model.compile(optimizer='adam', loss='mse', metrics=['mse'])
    
    return model


In [5]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

def create_resnet(input_shape, num_classes=None, task_type='classification'):
    base_model = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    if task_type == 'classification':
        outputs = Dense(num_classes, activation='softmax')(x)
        model = Model(inputs=base_model.input, outputs=outputs)
        model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    else:
        outputs = Dense(1, activation='linear')(x)
        model = Model(inputs=base_model.input, outputs=outputs)
        model.compile(optimizer='adam', loss='mse', metrics=['mse'])
    
    return model


In [6]:

input_shape = (224, 224, 3)
num_classes = y_pose_class.shape[1]


cnn_pose_model = create_cnn(input_shape, num_classes, task_type='classification')
cnn_pose_model.fit(X_train, y_train_pose, validation_data=(X_val, y_val_pose), epochs=10, batch_size=32)


lenet_pose_model = create_lenet(input_shape, num_classes, task_type='classification')
lenet_pose_model.fit(X_train, y_train_pose, validation_data=(X_val, y_val_pose), epochs=10, batch_size=32)


resnet_pose_model = create_resnet(input_shape, num_classes, task_type='classification')
resnet_pose_model.fit(X_train, y_train_pose, validation_data=(X_val, y_val_pose), epochs=10, batch_size=32)


NameError: name 'y_pose_class' is not defined

In [7]:

cnn_depth_model = create_cnn(input_shape, task_type='regression')
cnn_depth_model.fit(X_train, y_train_depth, validation_data=(X_val, y_val_depth), epochs=10, batch_size=32)


lenet_depth_model = create_lenet(input_shape, task_type='regression')
lenet_depth_model.fit(X_train, y_train_depth, validation_data=(X_val, y_val_depth), epochs=10, batch_size=32)


resnet_depth_model = create_resnet(input_shape, task_type='regression')
resnet_depth_model.fit(X_train, y_train_depth, validation_data=(X_val, y_val_depth), epochs=10, batch_size=32)


NameError: name 'create_cnn' is not defined

In [8]:
from sklearn.metrics import classification_report, confusion_matrix


pose_preds = np.argmax(cnn_pose_model.predict(X_test), axis=1)
pose_true = np.argmax(y_test_pose, axis=1)
print("CNN Pose Classification Report:")
print(classification_report(pose_true, pose_preds))




NameError: name 'cnn_pose_model' is not defined

In [9]:
from sklearn.metrics import mean_squared_error


depth_preds = cnn_depth_model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test_depth, depth_preds))
print(f"CNN Depth RMSE: {rmse}")




NameError: name 'cnn_depth_model' is not defined

In [10]:

# import pandas as pd

# results = {
#     "Model": ["CNN", "LeNet", "ResNet"],
#     "Pose Accuracy": [cnn_pose_acc, lenet_pose_acc, resnet_pose_acc],
#     "Depth RMSE": [cnn_depth_rmse, lenet_depth_rmse, resnet_depth_rmse],
# }

# df_results = pd.DataFrame(results)
# print(df_results)


NameError: name 'cnn_pose_acc' is not defined